In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta

In [2]:
from scipy import stats


### load data from DB

In [3]:
host = '192.168.1.27'
port = '3306'
db_name = 'quantdb_maria'
username = 'quantdb'
password = 'QuantDb2023!'

engine = create_engine("mysql+pymysql://" + username + ":" + password + "@" + host + ":" + port + "/" + db_name)
conn=engine.connect()

In [4]:
trading_dates=f"select unique(Dates) from fn_SSC"
dates=pd.read_sql(trading_dates, conn)

In [5]:
date.today() - timedelta(days=2)

datetime.date(2025, 5, 31)

In [6]:
period1='2011-01-01'
period2= date.today() - timedelta(days=3)

In [7]:
query_COM=f"select * from fn_COM where (Dates>='{period1}' and Dates<='{period2}')"
data_COM=pd.read_sql(query_COM, conn)

In [8]:
len(data_COM), len(data_COM[data_COM['KOSPI200YN']=='Y'])

(4250104, 709460)

### combine tables

In [9]:
query_CIA=f"select * from fn_CIA where Dates>='{period1}' and Dates<='{period2}'"
data_CIA=pd.read_sql(query_CIA, conn)

query_SSC=f"select * from fn_SSC where Dates>='{period1}' and Dates<='{period2}'"
data_SSC=pd.read_sql(query_SSC, conn)

query_IFRS=f"select * from fn_NFS_IFRS where Dates>='{period1}' and Dates<='{period2}'"
data_IFRS=pd.read_sql(query_IFRS, conn)

query_CON=f"select * from fn_CON where Dates>='{period1}' and Dates<='{period2}'"
data_CON=pd.read_sql(query_CON, conn)               

In [10]:
query_catalog=f"select * from Catalog_Table"
data_catalog=pd.read_sql(query_catalog, conn)

In [11]:
data_catalog

,Table_Name,Frequency,Dates,Item_Code,Item_Name_kor,Item_Name_eng,DB_col_name
0,COM,Annual,2017-03-24 00:00:00,CP30900010,시장 대표 업종(Hist.),Koscom Industry(Hist.),KC_Industry
1,COM,None,2017-03-24 00:00:00,CP10007100,FnGuide Industry Group 27,FnGuide Industry Group 27,FG_IndustryGroup27
2,COM,None,2017-03-24 00:00:00,CP10000650,FnGuide Sector (지수 편입 기준),FnGuide Sector (Index),FG_Sector_Idx
3,COM,None,2017-03-24 00:00:00,CP10000850,FnGuide Industry Group (지수 편입 기준),FnGuide Industry Group (Index),FG_IndustryGroup_Idx
4,COM,None,2017-03-24 00:00:00,CP10001050,FnGuide Industry (지수 편입 기준),FnGuide Industry (Index),FG_Industry_Idx
...,...,...,...,...,...,...,...
310,CON,2017(ANNUAL),2017-03-24 00:00:00,F730015336,"ROE (E3, 3M Chg)(%)","ROE (E3, 3M Chg)(%)",ROE_E3_3M_Chg_ic
311,CON,2017(ANNUAL),2017-03-24 00:00:00,F730015406,"ROE (E3, 6M Chg)(%)","ROE (E3, 6M Chg)(%)",ROE_E3_6M_Chg_ic
312,CON,2017(ANNUAL),2017-03-24 00:00:00,F730011820,SPS (E3)(원),SPS (E3)(Won),SPS_E3_ic
313,CON,2017(ANNUAL),2017-03-24 00:00:00,F730011495,판관비 (E3)(억원),Selling and Administrative Expenses (E3)(W 100mn),SAE_E3_ic


In [12]:
query_idx=f"select * from krx_idx"
krx_idx=pd.read_sql(query_idx, conn)

In [13]:
krx_idx.set_index(krx_idx.columns[0], inplace=True)

In [14]:
krx_idx

,코스피,코스피 200,코스피 100,코스피 50,코스닥 150,코스닥,코스피 200 동일가중,코스피 대형주,코스피 중형주,코스피 소형주,...,에너지,소재,산업재,경기소비재,필수소비재,의료,금융,IT,통신서비스,유틸리티
Dates,,,,,,,,,,,,,,,,,,,,,
2000-01-04,1059.04,133.66,NaN,NaN,NaN,2660.00,NaN,1000.00,1000.00,1000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,986.31,123.86,922.14,918.40,NaN,2629.50,NaN,923.61,970.47,992.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,960.79,120.80,900.82,898.72,NaN,2475.20,NaN,904.64,931.49,962.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,948.65,119.10,887.78,883.90,NaN,2276.60,NaN,890.32,938.66,966.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,987.24,124.11,927.12,924.83,NaN,2310.80,NaN,928.70,952.01,975.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-26,2644.40,352.11,2632.08,2385.39,1171.63,725.27,2378.37,2628.38,2975.49,2275.59,...,5351.65,3712.99,7981.96,4435.44,6152.68,22485.5,2860.45,7106.83,1133.54,1756.74
2025-05-27,2637.22,351.28,2624.08,2373.15,1179.67,727.11,2382.34,2619.50,2978.34,2274.11,...,5334.68,3663.48,8113.97,4406.12,6176.73,22729.7,2855.16,7035.29,1131.21,1703.86
2025-05-28,2670.15,356.36,2664.60,2414.48,1183.06,728.79,2405.60,2655.09,3000.78,2294.91,...,5630.55,3851.03,7960.72,4474.07,6127.68,22558.4,2854.70,7264.92,1124.57,1682.36


In [15]:
# pd.DatetimeIndex(com_data['Dates'].unique()).get_loc('2010-01-26')

### factor generating dates

In [16]:
# rebal_dates = []
# # Initialize a variable to keep track of the last month encountered
# last_month = None

# # Loop through the DataFrame rows and find the last date before month changes
# for row_idx, row in enumerate(given_dates):

#     current_month = row.month
#     if (current_month != last_month) and row_idx >0:
#         rebal_dates.append(given_dates[row_idx - 1])
#     last_month = current_month

# # Convert the list of last dates to a DataFrame
# rebal_dates_df = pd.DataFrame({'rebalancing_dates': rebal_dates})

# print(rebal_dates_df)

In [17]:
# # Group the data by bi-weekly intervals and select the last date of each interval
# def bi_weekly_end_date(group):
#     return group.iloc[-1]

# bi_weekly_end_dates=given_dates.to_frame().groupby(pd.Grouper(key=0, freq='2W')).apply(bi_weekly_end_date)
# weekly_end_dates=given_dates.to_frame().groupby(pd.Grouper(key=0, freq='1W')).last()

### preprocessing DB data

In [18]:
data_COM.shape, data_CIA.shape, data_SSC.shape, data_IFRS.shape, data_CON.shape

((4250104, 16), (4256471, 19), (4256495, 35), (4232772, 106), (4249592, 105))

In [19]:
data_COM.set_index(['Dates', 'Symbol'], inplace=True)
data_SSC.set_index(['Dates', 'Symbol'], inplace=True)
data_CIA.set_index(['Dates', 'Symbol'], inplace=True)
data_IFRS.set_index(['Dates', 'Symbol'], inplace=True)
data_CON.set_index(['Dates', 'Symbol'], inplace=True)

In [20]:
given_dates=data_COM.index.get_level_values(0)
given_dates=given_dates.unique()
weekly_end_dates=given_dates.to_frame().resample('w-sat').last()
weekly_end_dates=weekly_end_dates['Dates'].tolist()
weekly_end_dates=[x for x in weekly_end_dates if not pd.isnull(x)]

In [21]:
weekly_end_dates

[Timestamp('2011-01-07 00:00:00'),
 Timestamp('2011-01-14 00:00:00'),
 Timestamp('2011-01-21 00:00:00'),
 Timestamp('2011-01-28 00:00:00'),
 Timestamp('2011-02-01 00:00:00'),
 Timestamp('2011-02-11 00:00:00'),
 Timestamp('2011-02-18 00:00:00'),
 Timestamp('2011-02-25 00:00:00'),
 Timestamp('2011-03-04 00:00:00'),
 Timestamp('2011-03-11 00:00:00'),
 Timestamp('2011-03-18 00:00:00'),
 Timestamp('2011-03-25 00:00:00'),
 Timestamp('2011-04-01 00:00:00'),
 Timestamp('2011-04-08 00:00:00'),
 Timestamp('2011-04-15 00:00:00'),
 Timestamp('2011-04-22 00:00:00'),
 Timestamp('2011-04-29 00:00:00'),
 Timestamp('2011-05-06 00:00:00'),
 Timestamp('2011-05-13 00:00:00'),
 Timestamp('2011-05-20 00:00:00'),
 Timestamp('2011-05-27 00:00:00'),
 Timestamp('2011-06-03 00:00:00'),
 Timestamp('2011-06-10 00:00:00'),
 Timestamp('2011-06-17 00:00:00'),
 Timestamp('2011-06-24 00:00:00'),
 Timestamp('2011-07-01 00:00:00'),
 Timestamp('2011-07-08 00:00:00'),
 Timestamp('2011-07-15 00:00:00'),
 Timestamp('2011-07-

In [22]:
data_SSC['TradeStop'][data_SSC['TradeStop']=='정상']=0
data_SSC['TradeStop'][data_SSC['TradeStop']=='거래정지']=1

/tmp/ipykernel_2679670/1204221557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_SSC['TradeStop'][data_SSC['TradeStop']=='정상']=0
/tmp/ipykernel_2679670/1204221557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_SSC['TradeStop'][data_SSC['TradeStop']=='거래정지']=1


In [23]:
data_SSC[data_SSC.columns[2:]]=data_SSC[data_SSC.columns[2:]].astype(float)
data_CIA[data_CIA.columns[2:]]=data_CIA[data_CIA.columns[2:]].astype(float)

In [24]:
data_IFRS.loc[data_IFRS['LE_TQ']=='완전잠식']=99999
data_IFRS.loc[data_IFRS['LE_LQ']=='완전잠식']=99999
data_IFRS.loc[data_IFRS['LE_2LQ']=='완전잠식']=99999
data_IFRS.loc[data_IFRS['LE_3LQ']=='완전잠식']=99999

In [25]:
data_IFRS[data_IFRS.columns[2:]]=data_IFRS[data_IFRS.columns[2:]].astype(float)

In [26]:
data_CON[data_CON=='적지'].dropna(axis=0, how='all')

,,Name,BPS_E3_12M_Fwd,BPS_E3_12M_Tr,BPS_E3_24M_Fwd,BPSG_E3,EPS_E3_12M_Fwd,EPS_E3_12M_Tr,EPS_E3_24M_Fwd,EPS_E3_1M_Chg,EPS_E3_3M_Chg,...,PER_E3_6M_Chg_ic,PSR_E3_ic,ROA_E3_ic,ROE_E3_1M_Chg_ic,ROE_E3_3M_Chg_ic,ROE_E3_6M_Chg_ic,SPS_E3_ic,SAE_E3_ic,OI_E3_2yr_ic,OI_E3_ic
Dates,Symbol,,,,,,,,,,,,,,,,,,,,,
2025-01-02,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-03,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-06,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-07,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-08,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-09,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-10,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-13,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-14,A036420,NaN,NaN,NaN,NaN,적지,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data_CON=data_CON.mask(data_CON=='N/A', other=None)
data_CON.loc[data_CON['BPSG_E3']=='적전']=-999
data_CON.loc[data_CON['BPSG_E3']=='적지']=-999
data_CON.loc[data_CON['BPSG_E3']=='흑전']=999

data_CON[data_CON.columns[2:]]=data_CON[data_CON.columns[2:]].astype(float)

In [28]:
data_COM_nodup=data_COM.loc[~data_COM.index.duplicated(keep='first')].copy()
data_SSC_nodup=data_SSC.loc[~data_SSC.index.duplicated(keep='first')].copy()
data_CIA_nodup=data_CIA.loc[~data_CIA.index.duplicated(keep='first')].copy()
data_IFRS_nodup=data_IFRS.loc[~data_IFRS.index.duplicated(keep='first')].copy()
data_CON_nodup=data_CON.loc[~data_CON.index.duplicated(keep='first')].copy()

In [29]:
data_COM_nodup.shape, data_SSC_nodup.shape, data_CIA_nodup.shape, data_IFRS_nodup.shape, data_CON_nodup.shape

((4250104, 14), (4256495, 33), (4256471, 17), (4232772, 104), (4249592, 103))

In [30]:
temp_data=pd.concat([data_COM_nodup, data_CIA_nodup, data_SSC_nodup, data_IFRS_nodup, data_CON_nodup], axis=1)

In [31]:
temp_data.columns

Index(['Name', 'KC_Industry', 'FG_IndustryGroup27', 'FG_Sector_Idx',
       'FG_IndustryGroup_Idx', 'FG_Industry_Idx', 'AuditOpinion', 'Caution',
       'KSECapSize', 'KLCAIndustry',
       ...
       'PER_E3_6M_Chg_ic', 'PSR_E3_ic', 'ROA_E3_ic', 'ROE_E3_1M_Chg_ic',
       'ROE_E3_3M_Chg_ic', 'ROE_E3_6M_Chg_ic', 'SPS_E3_ic', 'SAE_E3_ic',
       'OI_E3_2yr_ic', 'OI_E3_ic'],
      dtype='object', length=271)

In [32]:
temp_data.dtypes

Name                     object
KC_Industry              object
FG_IndustryGroup27       object
FG_Sector_Idx            object
FG_IndustryGroup_Idx     object
                         ...   
ROE_E3_6M_Chg_ic        float64
SPS_E3_ic               float64
SAE_E3_ic               float64
OI_E3_2yr_ic            float64
OI_E3_ic                float64
Length: 271, dtype: object

In [33]:
temp_data=temp_data.dropna(how='all', axis=0)

In [34]:
temp_data=temp_data.sort_index(level='Dates')

In [35]:
factor_dict={
'sector': 'CP10000600',
'industry': 'CP10000800',
'mkt':'CP10000300',
'k2YN':'CP10000310',
'clsMnth':'CP10002200',
'adjPrice':'S410000700',
'trdYN':'S410002600',
'mgtYN':'S410002700',
'tmv':'S420002100',
'trdVol_20avg':'S41000630F',
'trdVol_60avg':'S41000640F',
'trdVol_120avg':'S41000650F',
'trA_20avg_spot':'S410007000',
'trA_60avg_spot':'S410007100',
'p_price_spot':'S41000060F',
'pHigh_spot':'S41000040F',
'pLow_spot':'S41000040F',
'retStdev_20_spot':'S410008600',
'retStdev_120_spot':'S410008800',
'senti_invTrst20':'CI20033022',
'senti_invTrst60':'CI20033023',
'senti_invTrst120':'CI20033024',
'senti_inst20':'CI20003022',
'senti_inst60':'CI20003023',
'senti_inst120':'CI20003024',
'senti_forg20':'CI20113022',
'senti_forg60':'CI20113023',
'senti_forg120':'CI20113024',
'epsConNmb':'FM10012110',
'epsF_spot':'FM30041100',
'epsT_spot':'FM30041200',
'bpsF_spot':'FM30041250',
'bpsT_spot':'FM30041260',
'salesF_spot':'FM30041500',
'salesT_spot':'FM30041530',
'earningsF_spot':'FM30041515',
'earningsT_spot':'FM30041545',
'cashFlowF_spot':'FM30041585',
'dbtRtQ_':'M000102001',
'totalEQ_0':'M000903001',
'salseQ_0':'M000904001',
'earningsQ_0':'M000906001',
'cashFlowQ_0':'M000909011',
'div_0':'M000706032',
}


### match raw factor codes with DB column

In [36]:
len(list(factor_dict.values()))

44

In [37]:
temp_cols=data_catalog[data_catalog['Item_Code'].isin(list(factor_dict.values()))]['DB_col_name']

In [38]:
temp_cols

9                   Market
10              KOSPI200YN
11             FiscalMonth
13     NetAmt_InvTrust_20D
14     NetAmt_InvTrust_60D
              ...         
141       Sales_E3_12M_Fwd
142        Sales_E3_12M_Tr
151          OI_E3_12M_Fwd
152           OI_E3_12M_Tr
170          CF_E3_12M_Fwd
Name: DB_col_name, Length: 68, dtype: object

In [39]:
cols=temp_data.columns[temp_data.columns.isin(temp_cols)]

In [40]:
len(cols)

68

In [41]:
temp_data[cols][temp_data['KOSPI200YN']=='Y']

Market KOSPI200YN FiscalMonth  NetAmt_InvTrust_20D  \
Dates      Symbol                                                       
2011-01-03 A000020    KSE          Y          12              25515.0   
           A000050    KSE          Y          12              19970.0   
           A000070    KSE          Y          12            -628575.0   
           A000100    KSE          Y          12              18519.0   
           A000120    KSE          Y          12              37361.0   
...                   ...        ...         ...                  ...   
2025-05-30 A402340    KSE          Y          12              42471.0   
           A450080    KSE          Y          12            -140429.0   
           A454910    KSE          Y          12              42635.0   
           A456040    KSE          Y          12               8191.0   
           A457190    KSE          Y          12              -8064.0   

                    NetAmt_InvTrust_60D  NetAmt_InvTrust_120D  \
Dates      Symbol                                               
2011-01-03 A000020              24112.0               23040.0   
           A000050             -46131.0               53507.0   
           A000070            -255594.0             -461158.0   
           A000100           -2765490.0               91228.0   
           A000120             181471.0               24131.0   
...                                 ...                   ...   
2025-05-30 A402340             314915.0             -692161.0   
           A450080           -1971870.0             1583220.0   
           A454910            -208577.0             -749878.0   
           A456040              24611.0              464757.0   
           A457190            -405928.0             -212932.0   

                    NetAmt_Inst_20D  NetAmt_Inst_60D  NetAmt_Inst_120D  \
Dates      Symbol                                                        
2011-01-03 A000020          37625.0          29939.0           19915.0   
           A000050          53877.0         -53901.0           40778.0   
           A000070        -367402.0        -442178.0        -1096950.0   
           A000100         392900.0       -5002870.0          180881.0   
           A000120        -216607.0        -970719.0        -1174430.0   
...                             ...              ...               ...   
2025-05-30 A402340        1432480.0        6605980.0        19018300.0   
           A450080        -620333.0       -4632900.0        -3938160.0   
           A454910       -1792600.0       -3913640.0        -7702010.0   
           A456040         -91513.0        -169775.0         -370728.0   
           A457190        -120973.0        -729626.0         3952090.0   

                    NetAmt_For_20D  ...  BPS_E3_12M_Fwd  BPS_E3_12M_Tr  \
Dates      Symbol                   ...                                  
2011-01-03 A000020         45483.0  ...            None            NaN   
           A000050         56468.0  ...            None            NaN   
           A000070        607258.0  ...        92661.66       89526.38   
           A000100        659916.0  ...       107542.74      103249.12   
           A000120       -435063.0  ...            None      102439.50   
...                            ...  ...             ...            ...   
2025-05-30 A402340       2356520.0  ...            None            NaN   
           A450080       -470569.0  ...            None            NaN   
           A454910       -634768.0  ...            None            NaN   
           A456040       -224559.0  ...            None            NaN   
           A457190        230950.0  ...            None            NaN   

                   EPS_E3_12M_Fwd  EPS_E3_12M_Tr  Num_EPS_Ests_E1  \
Dates      Symbol                                                   
2011-01-03 A000020            NaN            NaN              NaN   
           A000050            NaN            NaN              NaN   
           A000070   

### compute factors
#### - standardize in cross section (mean, std)
#### - winsorization
#### - 


In [42]:
data_catalog['DB_col_name']

0               KC_Industry
1        FG_IndustryGroup27
2             FG_Sector_Idx
3      FG_IndustryGroup_Idx
4           FG_Industry_Idx
               ...         
310        ROE_E3_3M_Chg_ic
311        ROE_E3_6M_Chg_ic
312               SPS_E3_ic
313               SAE_E3_ic
314            OI_E3_2yr_ic
Name: DB_col_name, Length: 315, dtype: object

### 1. find columns 

In [43]:
size=data_catalog[data_catalog['Item_Code']==factor_dict['tmv']]['DB_col_name']
div_ratio=data_catalog[data_catalog['Item_Code']==factor_dict['div_0']]['DB_col_name']

close=data_catalog[data_catalog['Item_Code']==factor_dict['p_price_spot']]['DB_col_name']
adjclose=data_catalog[data_catalog['Item_Code']==factor_dict['adjPrice']]['DB_col_name']

trdVol_20avg=data_catalog[data_catalog['Item_Code']==factor_dict['trdVol_20avg']]['DB_col_name']
trdVol_60avg=data_catalog[data_catalog['Item_Code']==factor_dict['trdVol_60avg']]['DB_col_name']

invTrst_20avg=data_catalog[data_catalog['Item_Code']==factor_dict['senti_invTrst20']]['DB_col_name']
invTrst_120avg=data_catalog[data_catalog['Item_Code']==factor_dict['senti_invTrst120']]['DB_col_name']

high=data_catalog[data_catalog['Item_Code']==factor_dict['pHigh_spot']]['DB_col_name']
low=data_catalog[data_catalog['Item_Code']==factor_dict['pLow_spot']]['DB_col_name']

stdev20=data_catalog[data_catalog['Item_Code']==factor_dict['retStdev_20_spot']]['DB_col_name']
stdev120=data_catalog[data_catalog['Item_Code']==factor_dict['retStdev_120_spot']]['DB_col_name']

earning_fwd=data_catalog[data_catalog['Item_Code']==factor_dict['earningsF_spot']]['DB_col_name']
earning_tr=data_catalog[data_catalog['Item_Code']==factor_dict['earningsT_spot']]['DB_col_name']

bps_fwd=data_catalog[data_catalog['Item_Code']==factor_dict['bpsF_spot']]['DB_col_name']
bps_tr=data_catalog[data_catalog['Item_Code']==factor_dict['bpsT_spot']]['DB_col_name']

eps_fwd=data_catalog[data_catalog['Item_Code']==factor_dict['epsF_spot']]['DB_col_name']
eps_tr=data_catalog[data_catalog['Item_Code']==factor_dict['epsT_spot']]['DB_col_name']

earning_Q=data_catalog[data_catalog['Item_Code']==factor_dict['earningsQ_0']]['DB_col_name']
sales_Q=data_catalog[data_catalog['Item_Code']==factor_dict['salseQ_0']]['DB_col_name']

dtoequity=data_catalog[data_catalog['Item_Code']==factor_dict['dbtRtQ_']]['DB_col_name']
equity=data_catalog[data_catalog['Item_Code']==factor_dict['totalEQ_0']]['DB_col_name']

sps_fwd=data_catalog[data_catalog['Item_Code']==factor_dict['salesF_spot']]['DB_col_name']
sps_tr=data_catalog[data_catalog['Item_Code']==factor_dict['salesT_spot']]['DB_col_name']

cf_Q=data_catalog[data_catalog['Item_Code']==factor_dict['cashFlowQ_0']]['DB_col_name']

cf_fwd=data_catalog[data_catalog['Item_Code']==factor_dict['cashFlowF_spot']]['DB_col_name']

netamt_for20=data_catalog[data_catalog['Item_Code']==factor_dict['senti_forg20']]['DB_col_name']
netamt_for60=data_catalog[data_catalog['Item_Code']==factor_dict['senti_forg60']]['DB_col_name']
netamt_for120=data_catalog[data_catalog['Item_Code']==factor_dict['senti_forg120']]['DB_col_name']
trd_amt20=data_catalog[data_catalog['Item_Code']==factor_dict['trA_20avg_spot']]['DB_col_name']
trd_amt60=data_catalog[data_catalog['Item_Code']==factor_dict['trA_60avg_spot']]['DB_col_name']


In [44]:
size

46    MktCap_Comm_Pref
Name: DB_col_name, dtype: object

In [45]:
factor_ls=['size', 
           'dividend', 
           'priceMomentum', 
           'trading', 
           'investSentiment', 
           'epsF_p', 'earningsOtmv', 
           '52H/p_Mom', '52L/p_Mom',
           'dtoequity', 'deltaDtoEquityYoY', 
           'stStevOLtStdev', 'Vol_20D', 'Vol_120D',
           'roeF_proxy', 'roeT_proxy', 'roeQ',
           'dEpsOp', 'rAdjdEpsOp',
           'roeFTMom', 'epsFMom', 'salseFMom', 'roeFchg_proxy', 'slsAdjEarningsYOY', 'sprsEarningsOearningsAvgPstdev',
           'epsFTMom', 'earningsFTMom', 'earningsYOY', 'salesYOY', 'cashFlowYOY', 
           'salesOtmv', 'cashFlowOtmv', 'equityOtmv', 'bpsF_p', 'bpsT_p', 'spsF_p', 'spsT_p', 'cfpsF_p',
           'senti_forg20', 'senti_forg60', 'senti_forg120', 'trA_20avg_spot', 'trA_60avg_spot']
len(factor_ls)

43

In [46]:
train_col = ['size', 'value', 'earningYield', 'quality', 'stability', 'dividend',
       'growth', 'momentum', 'adjMomentum', 'priceMomentum', 'priceReversal',
       'priceStdev', 'trading', 'investSentiment', 
       'senti_forg20', 'senti_forg60', 'senti_forg120', 'trA_20avg_spot', 'trA_60avg_spot']

In [47]:
len(cols), len(train_col)

(68, 19)

In [48]:
temp_data.index.get_level_values(0)

DatetimeIndex(['2011-01-03', '2011-01-03', '2011-01-03', '2011-01-03',
               '2011-01-03', '2011-01-03', '2011-01-03', '2011-01-03',
               '2011-01-03', '2011-01-03',
               ...
               '2025-05-30', '2025-05-30', '2025-05-30', '2025-05-30',
               '2025-05-30', '2025-05-30', '2025-05-30', '2025-05-30',
               '2025-05-30', '2025-05-30'],
              dtype='datetime64[ns]', name='Dates', length=4256495, freq=None)

In [49]:
len(weekly_end_dates)

751

In [50]:
data_ls=[]
skip_period=156 #156
data_generating_period=weekly_end_dates.copy()
normalize_data=False

for idx, compute_date in enumerate(data_generating_period[skip_period:]):
    
    # create snapshot df for past time
    eom_data=temp_data[cols][(temp_data['KOSPI200YN']=='Y') & (temp_data.index.get_level_values(0)==compute_date)]
    past1m_data=temp_data[cols][(temp_data.index.get_level_values(1).isin(eom_data.index.get_level_values(1))) & \
                                (temp_data.index.get_level_values(0)==data_generating_period[skip_period+idx-4])]
    past3m_data=temp_data[cols][(temp_data.index.get_level_values(1).isin(eom_data.index.get_level_values(1))) \
                                & (temp_data.index.get_level_values(0)==data_generating_period[skip_period+idx-12])]
    
    
    past12m_data=temp_data[cols][(temp_data.index.get_level_values(1).isin(eom_data.index.get_level_values(1))) \
                                 & (temp_data.index.get_level_values(0)==data_generating_period[skip_period+idx-48])]
    
    # when new stock is added, how to compute with different stocks
    eom_idx=set(eom_data.index.get_level_values(1))
    past1m_idx=set(past1m_data.index.get_level_values(1))
    past3m_idx=set(past3m_data.index.get_level_values(1))
    past12m_idx=set(past12m_data.index.get_level_values(1))
    common_idx=eom_idx.intersection(past1m_idx, past3m_idx, past12m_idx)

    eom_data=eom_data.loc[eom_data.index.get_level_values(1).isin(common_idx)]
    past1m_data=past1m_data.loc[past1m_data.index.get_level_values(1).isin(common_idx)]
    past3m_data=past3m_data.loc[past3m_data.index.get_level_values(1).isin(common_idx)]
    past12m_data=past12m_data.loc[past12m_data.index.get_level_values(1).isin(common_idx)]
    
    eom_data['size']=np.log(eom_data.loc[:, size])
    eom_data['dividend']=eom_data.loc[:, div_ratio]
    eom_data['priceMomentum']=(eom_data.loc[:, adjclose].values / past12m_data.loc[:, adjclose].values) - (eom_data.loc[:, adjclose].values / past1m_data.loc[:, adjclose].values)
    eom_data['trading']=eom_data.loc[:, trdVol_20avg].values / eom_data[trdVol_60avg].values
    eom_data['investSentiment']=(eom_data.loc[:, invTrst_20avg].values - eom_data.loc[:, invTrst_120avg].values)/eom_data.loc[:, size].values
    eom_data['epsF_p']=eom_data.loc[:, eps_fwd].values/eom_data[adjclose].values
    eom_data['earningsOtmv']=eom_data.loc[:, earning_Q[:4]].sum(1).to_frame().values/eom_data.loc[:, size].values
    eom_data['52H/p_Mom']=eom_data.loc[:, high].values/eom_data.loc[:, adjclose].values
    eom_data['52L/p_Mom']=eom_data.loc[:, low].values/eom_data.loc[:, adjclose].values
    eom_data['dtoequity']=eom_data.loc[:, dtoequity.iloc[0]].values/100
    eom_data['deltaDtoEquityYoY']=(past12m_data.loc[:, dtoequity.iloc[0]].values/100-eom_data.loc[:, dtoequity.iloc[0]].values/100) # its name is YOY but its data uses prev Q data
    eom_data['stStevOLtStdev']=eom_data.loc[:, stdev20].values / eom_data.loc[:, stdev120].values
    eom_data['roeF_proxy']=eom_data.loc[:, eps_fwd].values/eom_data.loc[:, bps_fwd].astype(float).values
    eom_data['roeT_proxy']=eom_data.loc[:, eps_tr].values / eom_data.loc[:, bps_tr].values
    eom_data['roeQ']=eom_data.loc[:, earning_Q.iloc[0]].values / eom_data.loc[:, equity.iloc[0]].values
    eom_data['dEpsOp']=(eom_data.loc[:, eps_fwd].values-past1m_data.loc[:, earning_fwd].values)/eom_data.loc[:, adjclose].values
    eom_data['rAdjdEpsOp']=eom_data.loc[:, 'dEpsOp'].to_frame().values/ (eom_data.loc[:, stdev20].values*np.sqrt(252))
    eom_data['roeFTMom']=eom_data.loc[:, 'roeF_proxy']-eom_data.loc[:, 'roeT_proxy']
    eom_data['epsFMom']=(eom_data.loc[:, eps_fwd].values - past1m_data.loc[:, eps_fwd].values)/past1m_data.loc[:, eps_fwd].values
    eom_data['salseFMom']=(eom_data.loc[:, sps_fwd].values - past1m_data.loc[:, sps_fwd].values)/past1m_data.loc[:, sps_fwd].values
    eom_data['roeFchg_proxy']=eom_data.loc[:, 'roeF_proxy'].to_frame().values-(past1m_data.loc[:, eps_fwd].values/past1m_data.loc[:, bps_fwd].astype(float).values)
    eom_data['earningsYOY']=(eom_data.loc[:, earning_Q.iloc[0]].values-past12m_data.loc[:, earning_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, earning_Q.iloc[0]].values) 
    eom_data['salesYOY']=(eom_data.loc[:, sales_Q.iloc[0]].values-past12m_data.loc[:, sales_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, sales_Q.iloc[0]].values) 
    eom_data['slsAdjEarningsYOY']=eom_data.loc[:, 'earningsYOY']-eom_data.loc[:, 'salesYOY']
    eom_data['sprsEarningsOearningsAvgPstdev']=(eom_data.loc[:, earning_Q.iloc[0]].to_frame().values-eom_data.loc[:, earning_Q[:4]].mean(1).to_frame().values)/(eom_data.loc[:, earning_Q[:4]].mean(1).to_frame().values+eom_data.loc[:, earning_Q[:4]].std(1).to_frame().values)
    eom_data['epsFTMom']=(eom_data.loc[:, eps_fwd].values-eom_data.loc[:, eps_tr].values)/np.abs(eom_data.loc[:, eps_tr].values)
    eom_data['earningsFTMom']=(eom_data.loc[:, earning_fwd].values-eom_data.loc[:, earning_tr].values)/np.abs(eom_data.loc[:, earning_tr].values)    
    eom_data['cashFlowYOY']=(eom_data.loc[:, cf_Q.iloc[0]].values-past12m_data.loc[:, cf_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, cf_Q.iloc[0]].values) 
    eom_data['salesOtmv']=eom_data.loc[:, sales_Q[:4]].sum(1).to_frame().values/eom_data.loc[:, size].values
    eom_data['cashFlowOtmv']=eom_data.loc[:, cf_Q].sum(1).to_frame().values/eom_data.loc[:, size].values
    eom_data['equityOtmv']=eom_data.loc[:, equity.iloc[0]].to_frame().values/eom_data.loc[:, size].values
    eom_data['bpsF_p']=eom_data.loc[:, bps_fwd].astype(float).values / eom_data[close].values
    eom_data['bpsT_p']=eom_data.loc[:, bps_tr].values / eom_data.loc[:, close].values
    eom_data['spsF_p']=eom_data.loc[:, sps_fwd].values / eom_data.loc[:, close].values
    eom_data['spsT_p']=eom_data.loc[:, sps_tr].values / eom_data.loc[:, close].values
    eom_data['cfpsF_p']=eom_data.loc[:, cf_fwd].values/eom_data.loc[:, close].values    
    eom_data['senti_forg20']=eom_data.loc[:, netamt_for20]
    eom_data['senti_forg60']=eom_data.loc[:, netamt_for60]
    eom_data['senti_forg120']=eom_data.loc[:, netamt_for120]
    eom_data['trA_20avg_spot']=eom_data.loc[:, trd_amt20]
    eom_data['trA_60avg_spot']=eom_data.loc[:, trd_amt60]

    
    if normalize_data:
        eom_data=eom_data.fillna(eom_data.quantile(.2))
        df=eom_data[factor_ls].copy()
        percentile_01=df.quantile(.01)
        percentile_99=df.quantile(.99)

        df = df.where(df > percentile_01, other=percentile_01, axis=1)
        df = df.where(df < percentile_99, other=percentile_99, axis=1)
        df_zscore=stats.zscore(df, axis=0)
        df_zscore[df_zscore>3]=3
        df_zscore[df_zscore<-3]=-3

        df_zscore['adjMomentum']=df_zscore[['dEpsOp', 'rAdjdEpsOp']].mean(1)
        df_zscore['earningYield']=df_zscore[['epsF_p', 'earningsOtmv']].mean(1)
        df_zscore['growth']=df_zscore[['epsFTMom', 'earningsFTMom', 'earningsYOY', 'salesYOY', 'cashFlowYOY']].mean(1)
        df_zscore['momentum']=df_zscore[['roeFTMom', 'epsFMom', 'salseFMom', 'roeFchg_proxy', 'slsAdjEarningsYOY', 'sprsEarningsOearningsAvgPstdev']].mean(1)
        df_zscore['priceReversal']=df_zscore[['52H/p_Mom', '52L/p_Mom']].mean(1)
        df_zscore['priceStdev']=df_zscore[['stStevOLtStdev', 'Vol_20D', 'Vol_120D']].mean(1)
        df_zscore['quality']=df_zscore[['roeF_proxy', 'roeT_proxy', 'roeQ']].mean(1)
        df_zscore['stability']=df_zscore[['dtoequity', 'deltaDtoEquityYoY']].mean(1)
        df_zscore['value']=df_zscore[['salesOtmv', 'cashFlowOtmv', 'equityOtmv', 'bpsF_p', 'bpsT_p', 'spsF_p', 'spsT_p', 'cfpsF_p']].mean(1)

        data_ls.append(df_zscore)

    else:
        eom_data['adjMomentum']=eom_data[['dEpsOp', 'rAdjdEpsOp']].mean(1)
        eom_data['earningYield']=eom_data[['epsF_p', 'earningsOtmv']].mean(1)
        eom_data['growth']=eom_data[['epsFTMom', 'earningsFTMom', 'earningsYOY', 'salesYOY', 'cashFlowYOY']].mean(1)
        eom_data['momentum']=eom_data[['roeFTMom', 'epsFMom', 'salseFMom', 'roeFchg_proxy', 'slsAdjEarningsYOY', 'sprsEarningsOearningsAvgPstdev']].mean(1)
        eom_data['priceReversal']=eom_data[['52H/p_Mom', '52L/p_Mom']].mean(1)
        eom_data['priceStdev']=eom_data[['stStevOLtStdev', 'Vol_20D', 'Vol_120D']].mean(1)
        eom_data['quality']=eom_data[['roeF_proxy', 'roeT_proxy', 'roeQ']].mean(1)
        eom_data['stability']=eom_data[['dtoequity', 'deltaDtoEquityYoY']].mean(1)
        eom_data['value']=eom_data[['salesOtmv', 'cashFlowOtmv', 'equityOtmv', 'bpsF_p', 'bpsT_p', 'spsF_p', 'spsT_p', 'cfpsF_p']].mean(1)
    
    data_ls.append(eom_data)

    # save as a file 
    
    # execute model


/tmp/ipykernel_2679670/1753552609.py:52: RuntimeWarning: divide by zero encountered in divide
  eom_data['earningsYOY']=(eom_data.loc[:, earning_Q.iloc[0]].values-past12m_data.loc[:, earning_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, earning_Q.iloc[0]].values)
/tmp/ipykernel_2679670/1753552609.py:53: RuntimeWarning: divide by zero encountered in divide
  eom_data['salesYOY']=(eom_data.loc[:, sales_Q.iloc[0]].values-past12m_data.loc[:, sales_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, sales_Q.iloc[0]].values)
/tmp/ipykernel_2679670/1753552609.py:58: RuntimeWarning: divide by zero encountered in divide
  eom_data['cashFlowYOY']=(eom_data.loc[:, cf_Q.iloc[0]].values-past12m_data.loc[:, cf_Q.iloc[0]].values)/np.abs(past12m_data.loc[:, cf_Q.iloc[0]].values)
/tmp/ipykernel_2679670/1753552609.py:52: RuntimeWarning: divide by zero encountered in divide
  eom_data['earningsYOY']=(eom_data.loc[:, earning_Q.iloc[0]].values-past12m_data.loc[:, earning_Q.iloc[0]].values)/np.abs(past12m_data.lo

In [51]:
len(common_idx)

199

In [52]:
data_df=pd.concat(data_ls)

In [53]:
data_df.index.get_level_values(0).unique()

DatetimeIndex(['2014-01-03', '2014-01-10', '2014-01-17', '2014-01-24',
               '2014-01-29', '2014-02-07', '2014-02-14', '2014-02-21',
               '2014-02-28', '2014-03-07',
               ...
               '2025-03-28', '2025-04-04', '2025-04-11', '2025-04-18',
               '2025-04-25', '2025-05-02', '2025-05-09', '2025-05-16',
               '2025-05-23', '2025-05-30'],
              dtype='datetime64[ns]', name='Dates', length=595, freq=None)

In [54]:
data_df['BPS_E3_12M_Fwd']=data_df['BPS_E3_12M_Fwd'].astype(float)

In [55]:
data_df.to_parquet('../data/output/raw_data/k200_weekly_factor_data_'+str(period1) + '_' +str(period2)+'v.1.parquet')

In [56]:
data_df[data_df.index.duplicated()]

,,Market,KOSPI200YN,FiscalMonth,NetAmt_InvTrust_20D,NetAmt_InvTrust_60D,NetAmt_InvTrust_120D,NetAmt_Inst_20D,NetAmt_Inst_60D,NetAmt_Inst_120D,NetAmt_For_20D,...,trA_60avg_spot,adjMomentum,earningYield,growth,momentum,priceReversal,priceStdev,quality,stability,value
Dates,Symbol,,,,,,,,,,,,,,,,,,,,,


In [57]:
temp_data2 = temp_data.drop_duplicates(keep='last')
temp_data2=temp_data2.loc[temp_data2['KOSPI200YN']=='Y', adjclose].unstack()

In [58]:
temp_data2.to_parquet('../data/output/raw_data/price_data_'+str(period1) + '_' + str(period2) +'.parquet')

In [59]:
temp_data2

AdjPrc                                                         \
Symbol     A000020 A000050  A000070   A000100  A000120  A000140   A000150   
Dates                                                                       
2011-01-03  5280.0  9199.0  73409.0   25339.0  93600.0  20200.0  126611.0   
2011-01-04  5300.0  9275.0  74447.0   25265.0  95600.0  20450.0  127013.0   
2011-01-05  5240.0  9047.0  74344.0   25265.0  95000.0  20600.0  125003.0   
2011-01-06  5110.0  8971.0  77978.0   25192.0  95400.0  20600.0  123797.0   
2011-01-07  5090.0  8971.0  76005.0   25118.0  95800.0  20600.0  124199.0   
...            ...     ...      ...       ...      ...      ...       ...   
2025-05-26     NaN     NaN      NaN  104200.0  80100.0      NaN  449000.0   
2025-05-27     NaN     NaN      NaN  105200.0  79800.0      NaN  474000.0   
2025-05-28     NaN     NaN      NaN  104500.0  80500.0      NaN  469500.0   
2025-05-29     NaN     NaN      NaN  105400.0  81200.0      NaN  510000.0   
2025-05-30     NaN     NaN      NaN  104300.0  80400.0      NaN  488000.0   

                                       ...                              \
Symbol       A000210 A000230  A000240  ...   A267260  A450080  A022100   
Dates                                  ...                               
2011-01-03  111596.0  9142.0  16440.0  ...       NaN      NaN      NaN   
2011-01-04  113855.0  9088.0  16465.0  ...       NaN      NaN      NaN   
2011-01-05  114307.0  8925.0  16787.0  ...       NaN      NaN      NaN   
2011-01-06  116114.0  8697.0  16515.0  ...       NaN      NaN      NaN   
2011-01-07  112952.0  8654.0  16168.0  ...       NaN      NaN      NaN   
...              ...     ...      ...  ...       ...      ...      ...   
2025-05-26   41750.0     NaN  16760.0  ...  389000.0  42950.0  21100.0   
2025-05-27   41500.0     NaN  16690.0  ...  392500.0  41050.0  20800.0   
2025-05-28   42800.0     NaN  16760.0  ...  379500.0  45550.0  21700.0   
2025-05-29   46100.0     NaN  19890.0  ...  383000.0  45500.0  21800.0   
2025-05-30   44250.0     NaN  20300.0  ...  379000.0  45100.0  22100.0   

                                                                             
Symbol      A005070  A066970  A454910  A457190   A002840   A278470  A456040  
Dates                                                                        
2011-01-03      NaN      NaN      NaN      NaN       NaN       NaN      NaN  
2011-01-04      NaN      NaN      NaN      NaN       NaN       NaN      NaN  
2011-01-05      NaN      NaN      NaN      NaN       NaN       NaN      NaN  
2011-01-06      NaN      NaN      NaN      NaN       NaN       NaN      NaN  
2011-01-07      NaN      NaN      NaN      NaN       NaN       NaN      NaN  
...             ...      ...      ...      ...       ...       ...      ...  
2025-05-26  30600.0  47800.0  47750.0  40200.0  179800.0  109900.0  53300.0  
2025-05-27  29700.0  48000.0  47050.0  39950.0  178500.0  118200.0  52500.0  
2025-05-28  32250.0  56100.0  47700.0  42350.0  182700.0  115300.0  53400.0  
2025-05-29  32300.0  56900.0  47800.0  41950.0  182700.0  114300.0  54600.0  
2025-05-30  32750.0  57200.0  47200.0  40500.0  182400.0  115300.0  55300.0  

[3545 rows x 366 columns]

In [60]:
temp_data2.columns=temp_data2.columns.get_level_values(1)

In [61]:
temp_data2=temp_data2.sort_index(level='Dates')

In [62]:
temp_data2.index

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2025-05-19', '2025-05-20', '2025-05-21', '2025-05-22',
               '2025-05-23', '2025-05-26', '2025-05-27', '2025-05-28',
               '2025-05-29', '2025-05-30'],
              dtype='datetime64[ns]', name='Dates', length=3545, freq=None)

In [63]:
krx_idx.to_parquet('../data/output/raw_data/krx_idx_' +str(period1) + '_'+ str(period2) + '.parquet')

In [64]:
krx_idx.index

DatetimeIndex(['2000-01-04', '2000-01-05', '2000-01-06', '2000-01-07',
               '2000-01-10', '2000-01-11', '2000-01-12', '2000-01-13',
               '2000-01-14', '2000-01-17',
               ...
               '2025-05-19', '2025-05-20', '2025-05-21', '2025-05-22',
               '2025-05-23', '2025-05-26', '2025-05-27', '2025-05-28',
               '2025-05-29', '2025-05-30'],
              dtype='datetime64[ns]', name='Dates', length=6266, freq=None)

In [65]:
a

NameError: name 'a' is not defined

In [ ]:
data_df.to_pickle('../data/output/raw_data/k200_weekly_factor_data_'+str(period1) + '_' +str(period2)+'.pkl')

In [ ]:
temp_data2.to_pickle('../data/output/pickle/price_data_'+str(period1) + '_' + str(period2) +'.pkl')

In [ ]:
krx_idx.to_pickle('../data/output/pickle/krx_idx_' +str(period1) + '_'+ str(period2) + '.pkl')

In [ ]:
# 'senti_forg20', 'senti_forg60', 'senti_forg120', 'trA_20avg_spot', 'trA_60avg_spot

In [ ]:
len(monthly_data)

In [ ]:
monthly_data_df=pd.concat(monthly_data)

In [ ]:
# monthly_data_df.to_parquet('./data/mlq_monthly_data_230601.parquet')

In [ ]:
monthly_data[1]

In [ ]:
eom_data[eps_fwd]

In [ ]:
temp_data[temp_data.index.duplicated()]

### extract price data
    

In [ ]:
temp_data.loc[temp_data['KOSPI200YN']=='Y', adjclose].unstack()